In [1]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform==1.88.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-04-ddc770beb944


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [40]:
# TODO: Define a function to add two numerical inputs and return the result.
# Keep the print statement within the function.
def add(a: int |float,b: int |float) -> int |float:
    """ 
    Adds 2 numbers together 
    Args:
    a: first number
    b: second number
    
    """
    print("Calling add function")
    return a+b


# TODO: Define a function to multiply two numerical inputs and return the result.
# Keep the print statement within the function.
def multiply(a: int |float,b: int |float) -> int |float:
    """ 
    Multiplies 2 numbers together 
    Args:
    a: first number
    b: second number
    
    """
    print("Calling multiply function")
    return a*b

# TODO: Create FunctionDeclarations for your functions

add_func = FunctionDeclaration.from_func(add)
multiply_func = FunctionDeclaration.from_func(multiply)
math_tool = Tool(
        function_declarations=[add_func,multiply_func ]
      )

In [8]:
# Initialize Gemini model
model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    generation_config = GenerationConfig(temperature=0),
    tools=[math_tool],
    system_instruction = [
              Part.from_text("""
    - Fulfill the user's instructions, including telling jokes.
    - If asked to add or multiply numbers, call the provided functions.
    - You may call one function after the other if needed.
    - Repeat the result to the user.
    """),
    ],
)
chat = model.start_chat()

In [72]:
def handle_response(response):
    """
    Handles the model's response, executing function calls if requested
    and recursively calling itself to process the final result.
    """
    # If there are no function calls, print the text response and exit.
    if not response.candidates[0].function_calls:
        print(f"{response.text}\n")
        return

    # If there is a function call, process it.
    function_calls = response.candidates[0].function_calls
    
    function_response_parts = []
    
    for function_call in function_calls:
        # Check which function is being called
        if function_call.name == "add":
            args = function_call.args
            result = add(a=args['a'], b=args['b'])
            api_response = {'result': result}
        
        elif function_call.name == "multiply":
            args = function_call.args
            result = multiply(a=args['a'], b=args['b'])
            api_response = {'result': result}
        
        else:
            # Should not happen with the current setup
            print(f"Unknown function call: {function_call.name}")
            continue

        # Create the function response part
        function_response_parts.append(
            Part.from_function_response(
                name=function_call.name,
                response=api_response
            )
        )

    # --- This is the key part ---
    # 1. Send the function response(s) back to the model.
    # print("Sending function result back to the model...")
    new_response = chat.send_message(
        function_response_parts
    )
    
    # 2. Make the recursive call to handle the model's new response.
    # This response will contain the natural language summary.
    handle_response(new_response)


In [73]:
response = chat.send_message("Tell me a joke?")
handle_response(response)

Why did the bicycle fall over? Because it was two tired!




In [74]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

Calling multiply function
112




In [75]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

Calling add function
7




In [76]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)    

Calling add function
Calling multiply function
There are 112 slices.




In [77]:
response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response)

Calling add function
There are 2 pizzas left.


